# Hands-On de Ánalise de Dados com Pandas

Aqui vamos praticar o que aprendemos nos outros notebooks de Pandas, utilizando essas técnicas para resolver algumas tarefas e ánalises. 

### Sobre os Dados

Utilizaremos as estatísticas dos voos de 2019 do Departamento de Transportes dos Estados Unidos (disponível [aqui](https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FMF&QO_fu146_anzr=Nv4%20Pn44vr45), sendo o arquivo `T100_MARKET_ALL_CARRIER.zip`). Cada linha contém informações sobre uma rota espeífica para uma determinada transportadora em um determinado mês (exemplo: JFK &rarr; LAX pela Delta Airlines em Janeiro). Existem 321.409 linhas e 41 colunas no dataset. Note que não é necessário extrair o arquivo zip para fazer a leitura utilizando `pd.read_csv()`.

#### Exercícios

##### 1. Leia os dados e converta o nomes das colunas para letras minúsculas para ser mais fácil de trabalhar com elas.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('DL_SelectFields.zip')
df.shape

(321388, 41)

##### 2. Quais são as colunas do dataset?

In [4]:
df.columns

Index(['PASSENGERS', 'FREIGHT', 'MAIL', 'DISTANCE', 'UNIQUE_CARRIER',
       'AIRLINE_ID', 'UNIQUE_CARRIER_NAME', 'UNIQUE_CARRIER_ENTITY', 'REGION',
       'CARRIER', 'CARRIER_NAME', 'CARRIER_GROUP', 'CARRIER_GROUP_NEW',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID',
       'ORIGIN', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS',
       'ORIGIN_STATE_NM', 'ORIGIN_COUNTRY', 'ORIGIN_COUNTRY_NAME',
       'ORIGIN_WAC', 'DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID',
       'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', 'DEST_STATE_ABR',
       'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_COUNTRY', 'DEST_COUNTRY_NAME',
       'DEST_WAC', 'YEAR', 'QUARTER', 'MONTH', 'DISTANCE_GROUP', 'CLASS',
       'DATA_SOURCE'],
      dtype='object')

##### 3. Quantas transportadores diferentes existem no dataset?

In [5]:
df.UNIQUE_CARRIER_NAME.nunique()

318

##### 4. Calcule o os valores totais das colunas `freight`, `mail`, e `passengers` para voos do Reino Unido para os Estados Unidos (United Kingdom e United States, respectivamente, no dataset).

In [6]:
df_q4 = df[['FREIGHT', 'MAIL', 'PASSENGERS','ORIGIN_COUNTRY_NAME', 'DEST_COUNTRY_NAME']]\
    .query("ORIGIN_COUNTRY_NAME == 'United Kingdom' and DEST_COUNTRY_NAME == 'United States'")\
    .groupby(['ORIGIN_COUNTRY_NAME', 'DEST_COUNTRY_NAME']).sum()

df_q4


,,FREIGHT,MAIL,PASSENGERS
ORIGIN_COUNTRY_NAME,DEST_COUNTRY_NAME,,,
United Kingdom,United States,903296879.0,29838395.0,10685614.0


##### 5. Ache as 5 transportadoras com maior distância mediana de rota (exemplo: para todos os pares origem-destino que cada portadora possui, encontre a distância mediana após remover duplicatas).

In [7]:
# Criando subset contendo o par origem-destino para avaliar as duplicatas
df_q5 = df[['DISTANCE', 'UNIQUE_CARRIER_NAME','ORIGIN', 'DEST']]
df_q5_no_dup = df_q5.drop_duplicates().groupby(['UNIQUE_CARRIER_NAME'])\
    .median().nlargest(n=5, columns='DISTANCE', keep='first')


In [8]:
df_q5_no_dup

,DISTANCE
UNIQUE_CARRIER_NAME,
Singapore Airlines Ltd.,8068.0
Cathay Pacific Airways Ltd.,8020.0
Qantas Airways Ltd.,7886.0
Longtail Aviation Ltd.,7855.5
National Aviation Company of India Limited d/b/a Air India,7798.0


##### 6. Ache a carga total transportada (mail + freight) e a distância média viajada para as 10 transportadoras que transportaram o maior número de cargas.

In [11]:
import numpy as np

df_q6 = df[['UNIQUE_CARRIER_NAME', 'FREIGHT', 'MAIL','DISTANCE']]\
    .assign(carga_total=lambda x: x.FREIGHT + x.MAIL)\
    .groupby('UNIQUE_CARRIER_NAME').agg(
        carga_total=('carga_total', 'sum'),
        distancia_media=('DISTANCE', np.mean)
    ).nlargest(10, columns='carga_total', keep='first')



In [12]:
df_q6

,carga_total,distancia_media
UNIQUE_CARRIER_NAME,,
Federal Express Corporation,1.270966e+10,1121.887981
United Parcel Service,9.173867e+09,1030.812650
Atlas Air Inc.,3.356847e+09,1900.987202
United Air Lines Inc.,1.577892e+09,1810.656058
American Airlines Inc.,1.353074e+09,1583.185048
Kalitta Air LLC,1.272180e+09,2519.580343
Polar Air Cargo Airways,1.199386e+09,3111.794118
Delta Air Lines Inc.,1.129524e+09,1612.688752
China Airlines Ltd.,8.370799e+08,5828.277778


##### 7. Quais as 10 transportadoras que transportaram o maior número de passageiros dos Estados Unidos para outro país?

##### 8. Para cada transportadora encontrada no exercício 7, ache o país destino mais popular fora os Estados Unidos.

##### 9. Para cada transportadora encontrada no exercício 7, ache o número total de passageiros que viajaram em voos internacionais para ou a partir dos destinos do exercício 8 ou dos Estados Unidos. Note que esse dataset só tem dados de voos com origem e/ou destino nos Estados Unidos.

##### 10. Entre quais duas cidades os passageiros mais viajaram? Lembre de contar para ambas as direções.

##### 11. Ache as 3 transportadoras com o maior número de voos com o par de cidades do exercício 10 como destino ou origem e calcule a porcentagem de passageiros contabilizados para essas cidades.

##### 12. Ache a porcentagem de viagens internacionais por país usando o total de passageiros em voos de classe F.

##### 13. Usando uma tabela cruzada, ache a porcentagem do total de passageiros que estão na classe F em voos internacionais entre cidades dos EUA e os países achados na questão 12 que usaram as transportadoras achadas na questão 11.

##### 14. Crie uma tabela pivotada mostrando o total de passageiros transportados entre cidades nos Estados Unidos e outros países pelas transportadoras identificadas no exercício 7. Selecione o top 10 das cidades dos EUA e o top 10 dos países internacionais desse resultado.

##### 15: Para o top 15 dos países internacionais, ache a porcentagem de passageiros de classe F viajando para ou a partir das top 10 cidades dos EUA, para as viagens internacionais (se apenas as cidades A, B e C voassem para Aruba, a soma da linha/coluna de Aruba seria 1). Faça a plotagem desses resultados como um heatmap.

# Referências

Link do repositório original - Git: https://github.com/stefmolin/pandas-workshop